In [1]:
!pip install -q ultralytics

In [2]:
# https://cocodataset.org/#download
# !wget http://images.cocodataset.org/zips/val2017.zip
# !unzip -qo val2017.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !unzip -qo annotations_trainval2017.zip

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CocoDetection
from tqdm.autonotebook import tqdm

<ipython-input-3-26863f95ef35>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
COCO_PATH = "val2017"
COCO_ANN_PATH = "annotations/instances_val2017.json"

In [5]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO

class COCODataset(torch.utils.data.Dataset):
    def __init__(self, root, annFile, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [6]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_Weights
from torchvision.transforms import functional as F
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import math
from sklearn.metrics import f1_score

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the collate function
def collate_fn(batch):
    return tuple(zip(*batch))

data_transform = transforms.Compose(
    [
        # transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)
coco_dataset = COCODataset(root=COCO_PATH, annFile=COCO_ANN_PATH, transforms=data_transform)


dataset_size = len(coco_dataset)
batch_size = 4
train_split = int(0.8 * dataset_size)
indices = list(range(dataset_size))

# todo toodo todo
train_sampler = SubsetRandomSampler(indices[:100])
val_sampler = SubsetRandomSampler(indices[1000:1100])
# train_sampler = SubsetRandomSampler(indices[:train_split])
# val_sampler = SubsetRandomSampler(indices[train_split:])

train_loader = DataLoader(coco_dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=lambda x: x)
val_loader = DataLoader(coco_dataset, batch_size=batch_size, sampler=val_sampler, collate_fn=lambda x: x)
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
num_classes = 91

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model = model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train the model
num_epochs = 2
for epoch in range(num_epochs):
    model.train()

    train_loss = []
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images = list(image[0].to(device) for image in batch)
        targets = [{k: v.to(device) for k, v in t[1].items()} for t in batch]
        new_targets = []
        new_images = []
        for target, image in zip(targets, images):
            if target['boxes'].shape[0] == 0:
                continue
            new_targets.append(target)
            new_images.append(image)
        loss_dict = model(new_images, new_targets)
        losses = sum(loss for loss in loss_dict.values())
        train_loss.extend(loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        # del targets, images, new_targets, new_images

    with torch.no_grad():
        model.eval()
        true_labels = []
        pred_labels = []
        for batch in tqdm(val_loader, desc=f"Val Epoch {epoch + 1}/{num_epochs}"):
            images = list(image[0].to(device) for image in batch)
            targets = [{k: v.to(device) for k, v in t[1].items()} for t in batch]
            new_targets = []
            new_images = []
            for target, image in zip(targets, images):
                if target['boxes'].shape[0] == 0:
                    continue
                new_targets.append(target)
                new_images.append(image)
            predictions = model(new_images, new_targets)

            for i in range(len(predictions)):
                pred = predictions[i]['labels'].cpu().tolist()
                target = targets[i]['labels'].cpu().tolist()
                if len(pred) < len(target):
                    pred.extend([0] * (len(target) - len(pred)))
                elif len(pred) > len(target):
                    # a lot noisy predictions with low score
                    target.extend([0] * (len(pred) - len(target)))
                pred_labels.extend(pred)
                true_labels.extend(target)

        print("F1 Score: ", f1_score(true_labels, pred_labels, average='macro'), "Train Loss: ", sum(train_loss)/len(train_loss))

    lr_scheduler.step()

# Save the fine-tuned model
torch.save(model.state_dict(), "fine_tuned_model.pth")

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


Epoch 1/2:   0%|          | 0/25 [00:00<?, ?it/s]

Val Epoch 1/2:   0%|          | 0/25 [00:00<?, ?it/s]

F1 Score:  0.09652694610778444 Train Loss:  tensor(0.3708, device='cuda:0')


Epoch 2/2:   0%|          | 0/25 [00:00<?, ?it/s]

Val Epoch 2/2:   0%|          | 0/25 [00:00<?, ?it/s]

F1 Score:  0.1213680387409201 Train Loss:  tensor(0.1574, device='cuda:0')
